In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
# Generar números de 8 bits
X = np.array([list(map(int, list(bin(i)[2:].zfill(9)))) for i in range(512)])
y = np.array([1 if i & 1 == 1 else 0 for i in range(512)])

In [7]:
# Imprimir números de entrenamiento y sus etiquetas
print("Números de entrenamiento y sus etiquetas:")
for i in range(len(X)):
    print(f"Número: {X[i]}, Etiqueta: {y[i]}")

Números de entrenamiento y sus etiquetas:
Número: [0 0 0 0 0 0 0 0 0], Etiqueta: 0
Número: [0 0 0 0 0 0 0 0 1], Etiqueta: 1
Número: [0 0 0 0 0 0 0 1 0], Etiqueta: 0
Número: [0 0 0 0 0 0 0 1 1], Etiqueta: 1
Número: [0 0 0 0 0 0 1 0 0], Etiqueta: 0
Número: [0 0 0 0 0 0 1 0 1], Etiqueta: 1
Número: [0 0 0 0 0 0 1 1 0], Etiqueta: 0
Número: [0 0 0 0 0 0 1 1 1], Etiqueta: 1
Número: [0 0 0 0 0 1 0 0 0], Etiqueta: 0
Número: [0 0 0 0 0 1 0 0 1], Etiqueta: 1
Número: [0 0 0 0 0 1 0 1 0], Etiqueta: 0
Número: [0 0 0 0 0 1 0 1 1], Etiqueta: 1
Número: [0 0 0 0 0 1 1 0 0], Etiqueta: 0
Número: [0 0 0 0 0 1 1 0 1], Etiqueta: 1
Número: [0 0 0 0 0 1 1 1 0], Etiqueta: 0
Número: [0 0 0 0 0 1 1 1 1], Etiqueta: 1
Número: [0 0 0 0 1 0 0 0 0], Etiqueta: 0
Número: [0 0 0 0 1 0 0 0 1], Etiqueta: 1
Número: [0 0 0 0 1 0 0 1 0], Etiqueta: 0
Número: [0 0 0 0 1 0 0 1 1], Etiqueta: 1
Número: [0 0 0 0 1 0 1 0 0], Etiqueta: 0
Número: [0 0 0 0 1 0 1 0 1], Etiqueta: 1
Número: [0 0 0 0 1 0 1 1 0], Etiqueta: 0
Número: [0 0 0 

In [8]:
# Convertir los datos de entrada a tensores de PyTorch
X = torch.from_numpy(X).float()
y = torch.from_numpy(y).view(-1, 1).float()


In [9]:
# Definir el modelo de regresión logística
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(9, 20) # 8 entradas y 20 salida
        self.linear2 = nn.Linear(20, 10)
        self.linear3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = torch.sigmoid(self.linear3(x))
        return x

In [10]:
# Crear una instancia del modelo
model = LogisticRegression()

# Definir la función de pérdida y el optimizador
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [11]:
# Entrenar el modelo
num_epochs = 10000
for epoch in range(num_epochs):
    # Reiniciar gradientes
    optimizer.zero_grad()
    # Forward pass
    outputs = model(X)
    # Calcular la pérdida
    loss = criterion(outputs, y)
    # Backward pass
    loss.backward()
    # Actualizar los pesos
    optimizer.step()

    # Imprimir la pérdida en cada 1000 épocas
    if (epoch + 1) % 1000 == 0:
        print(f'Época [{epoch + 1}/{num_epochs}], Pérdida: {loss.item():.4f}')


Época [1000/10000], Pérdida: 0.6763
Época [2000/10000], Pérdida: 0.0385
Época [3000/10000], Pérdida: 0.0076
Época [4000/10000], Pérdida: 0.0039
Época [5000/10000], Pérdida: 0.0026
Época [6000/10000], Pérdida: 0.0019
Época [7000/10000], Pérdida: 0.0015
Época [8000/10000], Pérdida: 0.0012
Época [9000/10000], Pérdida: 0.0010
Época [10000/10000], Pérdida: 0.0009


In [12]:
# Evaluar el modelo entrenado
with torch.no_grad():
    predicted = model(X)
    predicted = (predicted > 0.5).float() # Aplicar umbral de 0.5 para obtener etiquetas binarias
    accuracy = (predicted == y).float().mean().item()
    print(f'Precisión del modelo entrenado: {accuracy:.4f}')

Precisión del modelo entrenado: 1.0000


In [13]:
# Generar algunos números de prueba para realizar predicciones
X_test = torch.tensor([[1, 0, 0, 1, 0, 1, 1, 0,1], [0, 1, 1, 0, 1, 0, 0, 1,1], [1, 1, 1, 1, 1, 1, 1, 1,1]], dtype=torch.float32)

# Realizar predicciones con el modelo entrenado
with torch.no_grad():
    predicted_test = model(X_test)
    predicted_test = (predicted_test > 0.5).float() # Aplicar umbral de 0.5 para obtener etiquetas binarias

# Imprimir las predicciones
for i in range(X_test.size(0)):
    print(f'Número de prueba {i + 1}: {X_test[i].numpy()} -> Predicción: {predicted_test[i].item()}')


Número de prueba 1: [1. 0. 0. 1. 0. 1. 1. 0. 1.] -> Predicción: 1.0
Número de prueba 2: [0. 1. 1. 0. 1. 0. 0. 1. 1.] -> Predicción: 1.0
Número de prueba 3: [1. 1. 1. 1. 1. 1. 1. 1. 1.] -> Predicción: 1.0


In [14]:
# Guardar los pesos del modelo en un archivo .pth
torch.save(model.state_dict(), 'indicatrizB.pth')
print("Los pesos del modelo se han guardado")

Los pesos del modelo se han guardado
